In [1]:
pip install -U -q tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 9.8 MB/s eta 0:00:00


## Imports

In [2]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 10

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import tensorflow as tf
import pathlib

# Define the paths to the training and testing data folders
train_data_path = "/content/drive/MyDrive/PreprocessSeg"
test_data_path = "/content/drive/MyDrive/PreprocessSeg"

# Use TensorFlow's image data generator to load the images and labels from the folders
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_data_generator.flow_from_directory(directory=train_data_path,
                                                      target_size=(128, 128),
                                                      batch_size=32,
                                                      class_mode='categorical',
                                                      shuffle=True)

test_data = test_data_generator.flow_from_directory(directory=test_data_path,
                                                    target_size=(128,128),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=False)

# Print the shapes of the data and labels
print("Training data shape:", train_data.image_shape)
print("Training labels shape:", train_data.classes.shape)
print("Testing data shape:", test_data.image_shape)
print("Testing labels shape:", test_data.classes.shape)

# Assign the data and labels to variables x_train, y_train, x_test, and y_test
x_train, y_train = train_data.next()
x_test, y_test = test_data.next()


Found 1921 images belonging to 4 classes.
Found 1921 images belonging to 4 classes.
Training data shape: (128, 128, 3)
Training labels shape: (1921,)
Testing data shape: (128, 128, 3)
Testing labels shape: (1921,)


In [18]:
#(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
val_split = 0.1

val_indices = int(len(x_train) * val_split)
new_x_train, new_y_train = x_train[val_indices:], y_train[val_indices:]
x_val, y_val = x_train[:val_indices], y_train[:val_indices]

print(f"Training data samples: {len(new_x_train)}")
print(f"Validation data samples: {len(x_val)}")
print(f"Test data samples: {len(x_test)}")

Training data samples: 29
Validation data samples: 3
Test data samples: 32


In [19]:
import tensorflow as tf
import numpy as np

# Assume y_train and y_test have a shape of [?, 4]
# Convert them to integer labels
y_train_int = np.argmax(y_train, axis=1)
y_test_int = np.argmax(y_test, axis=1)

# Now you can convert them to one-hot encoded vectors with num_classes=4
y_train = tf.keras.utils.to_categorical(y_train_int, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test_int, num_classes=4)

y_train = y_train.squeeze()
y_test = y_test.squeeze()


## Prepare `tf.data.Dataset` objects

Our data augmentation pipeline is different from what the authors used for the CIFAR-10
dataset, which is fine for the purpose of the example.

In [ ]:
image_size = 176
auto = tf.data.AUTOTUNE

data_augmentation = keras.Sequential(
    [layers.RandomCrop(image_size, image_size), layers.RandomFlip("horizontal"),],
    name="data_augmentation",
)


def make_datasets(images, labels, is_train=False):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if is_train:
        dataset = dataset.shuffle(batch_size * 10)
    dataset = dataset.batch(batch_size)
    if is_train:
        dataset = dataset.map(
            lambda x, y: (data_augmentation(x), y), num_parallel_calls=auto
        )
    return dataset.prefetch(auto)


train_dataset = make_datasets(new_x_train, new_y_train, is_train=True)
val_dataset = make_datasets(x_val, y_val)
test_dataset = make_datasets(x_test, y_test)

## ConvMixer utilities

The following figure (taken from the original paper) depicts the ConvMixer model:

![](https://i.imgur.com/yF8actg.png)

ConvMixer is very similar to the MLP-Mixer, model with the following key
differences:

* Instead of using fully-connected layers, it uses standard convolution layers.
* Instead of LayerNorm (which is typical for ViTs and MLP-Mixers), it uses BatchNorm.

Two types of convolution layers are used in ConvMixer. **(1)**: Depthwise convolutions,
for mixing spatial locations of the images, **(2)**: Pointwise convolutions (which follow
the depthwise convolutions), for mixing channel-wise information across the patches.
Another keypoint is the use of *larger kernel sizes* to allow a larger receptive field.

In [20]:

def activation_block(x):
    x = layers.Activation("gelu")(x)
    return layers.BatchNormalization()(x)


def conv_stem(x, filters: int, patch_size: int):
    x = layers.Conv2D(filters, kernel_size=patch_size, strides=patch_size)(x)
    return activation_block(x)


def conv_mixer_block(x, filters: int, kernel_size: int):
    # Depthwise convolution.
    x0 = x
    x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
    x = layers.Add()([activation_block(x), x0])  # Residual.

    # Pointwise convolution.
    x = layers.Conv2D(filters, kernel_size=1)(x)
    x = activation_block(x)

    return x


def get_conv_mixer_256_8(
    image_size=128, filters=256, depth=8, kernel_size=5, patch_size=2, num_classes=4
):
    """ConvMixer-256/8: https://openreview.net/pdf?id=TVHS5Y4dNvM.
    The hyperparameter values are taken from the paper.
    """
    inputs = keras.Input((image_size, image_size, 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Extract patch embeddings.
    x = conv_stem(x, filters, patch_size)

    # ConvMixer blocks.
    for _ in range(depth):
        x = conv_mixer_block(x, filters, kernel_size)

    # Classification block.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)


## Model training and evaluation utility

In [23]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy", # use categorical crossentropy loss
        metrics=["accuracy"],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        new_x_train,  # use new_x_train instead of x_train
        new_y_train,  # use new_y_train instead of y_train
        epochs=num_epochs,
        validation_split=0.2,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(test_data)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, model


## Train and evaluate model

In [24]:
conv_mixer_model = get_conv_mixer_256_8()
history, conv_mixer_model = run_experiment(conv_mixer_model)

Epoch 1/10
1/1 [==============================] - 47s 47s/step - loss: 1.3963 - accuracy: 0.2174 - val_loss: 1.3769 - val_accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 33s 33s/step - loss: 1.3130 - accuracy: 0.4783 - val_loss: 1.3682 - val_accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 33s 33s/step - loss: 1.1810 - accuracy: 0.6087 - val_loss: 1.3597 - val_accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 34s 34s/step - loss: 1.0577 - accuracy: 0.6087 - val_loss: 1.3510 - val_accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 32s 32s/step - loss: 0.9434 - accuracy: 0.6957 - val_loss: 1.3424 - val_accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 31s 31s/step - loss: 0.8392 - accuracy: 0.7391 - val_loss: 1.3340 - val_accuracy: 0.3333
Epoch 7/10
1/1 [==============================] - 34s 34s/step - loss: 0.7231 - accuracy: 0.7826 - val_loss: 1.3265 - val_accuracy: 0.3333
Epoch 8/10
1/1 [===========